# Parameters

In [ ]:
# RaspberryPi connect library
import RaspberryPi

# Oscilloscope
from lecroy_python3 import *

# 추가 import
import os
from tqdm.auto import tqdm
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

# RaspberryPi parameters
HOST = '192.168.137.233'
ID = 'sungbin'
PW = '1234'
PORT = 22
AESFILE = 'raspberry/TinyAES'

# Oscilloscope
oscilloIP = '192.168.0.5'
traceLen  = int(25e4)

# SaveFile
resultFolderName = "C:/Users/FI 4/Desktop/raspberryPiCode/라즈베리파이_20240923/파형수집"

# 수집할 파형 갯수
Tracenumber = 100000 # 위치 당 수집할 파형 갯수

# Raspberry Pi connect

In [ ]:
Rasp = RaspberryPi.RaspberryPi(HOST = HOST, ID = ID, PW = PW, PORT = PORT, AESFILENAME = AESFILE)

# Oscillo setting

In [ ]:
oscillo = LeCroy()
oscillo.connect(oscilloIP)

# 암호문 검증

In [ ]:
SBox    = np.array([0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76, 0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0, 0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15, 0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75, 0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84, 0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf, 0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8, 0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2, 0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73, 0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb, 0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79, 0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08, 0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a, 0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e, 0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf, 0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16], dtype=np.uint8)
masterkey = np.array([0x59, 0x6f, 0x75, 0x20, 0x52, 0x20, 0x53, 0x43, 0x41, 0x20, 0x4d, 0x61, 0x73, 0x74, 0x65, 0x72], dtype = np.uint8)

def AESxtimes(inputData):
    return ((inputData << 1) & 0xFF) ^ ((inputData >> 7) * 0x1b)
def AESshiftRows(state):
    outputState = np.zeros(16, dtype=np.uint8)
    for i in range(16):
        outputState[i] = state[(i + 4 * (i % 4)) % 16]
    return outputState
def AESmixColumns(state):
    outputState = np.zeros(16, dtype=np.uint8)
    for i in range(0, 16, 4):
        outputState[i    ] = AESxtimes(state[i  ] ^ state[i+1]) ^ state[i+1] ^ state[i+2] ^ state[i+3]
        outputState[i + 1] = AESxtimes(state[i+1] ^ state[i+2]) ^ state[i+2] ^ state[i+3] ^ state[i  ]
        outputState[i + 2] = AESxtimes(state[i+2] ^ state[i+3]) ^ state[i+3] ^ state[i  ] ^ state[i+1]
        outputState[i + 3] = AESxtimes(state[i+3] ^ state[i  ]) ^ state[i  ] ^ state[i+1] ^ state[i+2]
    return outputState     
def AESkeyExpansion(masterKey):
    masterKey = np.array(masterKey, dtype=np.uint8)
    
    rcon        = 1
    roundKey    = np.zeros((11, 16), dtype=np.uint8)
    roundKey[0] = masterKey.copy()
    for roundIndex in range(1, 11):
        roundKey[roundIndex][0] = roundKey[roundIndex - 1][0] ^ SBox[roundKey[roundIndex - 1][13]] ^ rcon
        roundKey[roundIndex][1] = roundKey[roundIndex - 1][1] ^ SBox[roundKey[roundIndex - 1][14]]
        roundKey[roundIndex][2] = roundKey[roundIndex - 1][2] ^ SBox[roundKey[roundIndex - 1][15]]
        roundKey[roundIndex][3] = roundKey[roundIndex - 1][3] ^ SBox[roundKey[roundIndex - 1][12]]
        for i in range(4, 16, 4):
            roundKey[roundIndex][i    ] = roundKey[roundIndex][i - 4] ^ roundKey[roundIndex - 1][i    ]
            roundKey[roundIndex][i + 1] = roundKey[roundIndex][i - 3] ^ roundKey[roundIndex - 1][i + 1]
            roundKey[roundIndex][i + 2] = roundKey[roundIndex][i - 2] ^ roundKey[roundIndex - 1][i + 2]
            roundKey[roundIndex][i + 3] = roundKey[roundIndex][i - 1] ^ roundKey[roundIndex - 1][i + 3]
        rcon = AESxtimes(rcon)
    return roundKey
def AES(plaintext, masterKey):
    plaintext = np.array(plaintext, dtype=np.uint8)
    masterKey = np.array(masterKey, dtype=np.uint8)
    roundKey = AESkeyExpansion(masterKey)

    state = plaintext ^ roundKey[0]
    for roundIndex in range(1, 10):
        state = SBox[state]
        state = AESshiftRows(state)
        state = AESmixColumns(state)
        state ^= roundKey[roundIndex]
    state = SBox[state]
    state = AESshiftRows(state)
    state ^= roundKey[10]
    
    return state

# 파형 수집

In [ ]:
os.makedirs('{}/trace{}'.format(resultFolderName, Tracenumber), exist_ok=True)
plains  = np.zeros((Tracenumber, 16), dtype=np.uint8)
ciphers = np.zeros((Tracenumber, 16), dtype=np.uint8)
traces = np.zeros((Tracenumber, traceLen), dtype=np.float32)
for traceIndex in tqdm(range(traceNum)):
    plains[traceIndex] = np.random.randint(low=0, high=256, size=16, dtype=np.uint8)
    Rasp.sendPT(plains[traceIndex])
    Rasp.readCT(ciphers[traceIndex])
    if (ciphers[traceIndex] != AES(plains[traceIndex], rightKey)).any():
        print('ERROR!! {} trace'.format(traceIndex))
        print(plains[traceIndex])
        print(ciphers[traceIndex])
        print(AES(plains[traceIndex], rightKey))
    traces[traceIndex] = oscillo.getDataFloats(channel='C2')[:traceLen].astype(np.float32)
    
np.save('{}/trace/trace.npy'.format(resultFolderName, traceIndex), traces)
np.save('{}/trace/plain.npy'.format(resultFolderName), plains)
np.save('{}/trace/cipher.npy'.format(resultFolderName), ciphers)